# Infant Respiration Estimation - Vollständige Pipeline

Dieses Notebook enthält alle erforderlichen Schritte zum Ausführen des Infant Respiration Estimation Projekts.

## Überblick

Dieses Projekt implementiert automatische Atmungsschätzung bei Säuglingen aus Videos unter Verwendung von Deep Learning-Methoden, insbesondere dem **AIRFlowNet**-Modell.

**Paper:** [Automatic Infant Respiration Estimation from Video: A Deep Flow-based Algorithm and a Novel Public Benchmark](https://arxiv.org/pdf/2307.13110.pdf)

### Unterstützte Modelle:
- **VIRENet** (AIRFlowNet)
- DeepPhys
- EfficientPhys
- TS-CAN

### Datasets:
- **AIR-125**: 125 annotierte Säuglingsvideos
- **COHFACE**: Erwachsenen-Atmungsdatensatz

## 1. Umgebungsüberprüfung und Setup

In [ ]:
import sys
import os
import subprocess

# Prüfe Python-Version
print(f"Python Version: {sys.version}")
print(f"Current Working Directory: {os.getcwd()}")

# Prüfe CUDA-Verfügbarkeit
try:
    import torch
    print(f"\nPyTorch Version: {torch.__version__}")
    print(f"CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"Number of GPUs: {torch.cuda.device_count()}")
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("\nPyTorch ist noch nicht installiert.")

## 2. Installation der Abhängigkeiten

**Hinweis:** Dieser Schritt kann einige Minuten dauern.

In [ ]:
# Installation der erforderlichen Pakete
!pip install -q h5py==2.10.0
!pip install -q yacs==0.1.8
!pip install -q scipy==1.5.2
!pip install -q pandas==1.1.5
!pip install -q scikit-image==0.17.2
!pip install -q numpy==1.22.0
!pip install -q matplotlib==3.1.2
!pip install -q opencv-python==4.5.2.54
!pip install -q PyYAML==6.0
!pip install -q scikit-learn==1.0.2
!pip install -q tensorboardX==2.4.1
!pip install -q tqdm==4.64.0
!pip install -q mat73==0.59

# Für PyTorch (falls noch nicht installiert)
# !pip install torch==1.12.1 torchvision==0.13.1 torchaudio==0.12.1

print("\nInstallation abgeschlossen!")

## 3. Dataset Download und Extraktion

Der AIR-125 Dataset wird von der Northeastern University heruntergeladen und extrahiert.

In [ ]:
import urllib.request
import zipfile
from pathlib import Path
from tqdm.notebook import tqdm

# Definiere Pfade
dataset_url = "https://coe.northeastern.edu/Research/AClab/AIR-125/AIR.zip"
download_path = Path("./data/AIR.zip")
extract_path = Path("./data/AIR")

# Erstelle Verzeichnis
download_path.parent.mkdir(parents=True, exist_ok=True)

# Download mit Fortschrittsanzeige
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

if not download_path.exists():
    print(f"Downloading AIR-125 dataset from {dataset_url}...")
    print("Dies kann mehrere Minuten dauern, je nach Internetgeschwindigkeit.")
    
    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc='AIR.zip') as t:
        urllib.request.urlretrieve(dataset_url, download_path, reporthook=t.update_to)
    print(f"\nDownload abgeschlossen: {download_path}")
else:
    print(f"Dataset bereits heruntergeladen: {download_path}")

# Extrahiere ZIP-Datei
if not extract_path.exists():
    print(f"\nExtrahiere Dataset nach {extract_path}...")
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path.parent)
    print("Extraktion abgeschlossen!")
else:
    print(f"Dataset bereits extrahiert: {extract_path}")

# Zeige Dataset-Struktur
print("\nDataset-Struktur:")
for item in sorted(extract_path.rglob('*'))[:10]:
    print(f"  {item.relative_to(extract_path.parent)}")
print("  ...")

## 4. Dataset-Inspektion

Lass uns einen Blick auf die Dataset-Struktur und einige Sample-Daten werfen.

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Zähle Dateien im Dataset
dataset_path = Path("./data/AIR")

if dataset_path.exists():
    video_files = list(dataset_path.rglob('*.avi')) + list(dataset_path.rglob('*.mp4'))
    mat_files = list(dataset_path.rglob('*.mat'))
    
    print(f"Anzahl Video-Dateien: {len(video_files)}")
    print(f"Anzahl MAT-Dateien (Annotationen): {len(mat_files)}")
    
    if video_files:
        print("\nBeispiel Video-Dateien:")
        for vf in video_files[:5]:
            print(f"  - {vf.name}")
    
    # Versuche ein Video zu laden und ein Frame anzuzeigen
    if video_files:
        sample_video = video_files[0]
        print(f"\nLade Sample-Video: {sample_video.name}")
        
        cap = cv2.VideoCapture(str(sample_video))
        ret, frame = cap.read()
        
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 6))
            plt.imshow(frame_rgb)
            plt.title(f"Sample Frame from {sample_video.name}")
            plt.axis('off')
            plt.show()
            
            # Video-Informationen
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
            print(f"\nVideo-Informationen:")
            print(f"  - FPS: {fps}")
            print(f"  - Frames: {frame_count}")
            print(f"  - Auflösung: {width}x{height}")
            print(f"  - Dauer: {frame_count/fps:.2f} Sekunden")
        
        cap.release()
else:
    print(f"Dataset-Pfad nicht gefunden: {dataset_path}")
    print("Bitte führe Zelle 3 aus, um das Dataset herunterzuladen.")

## 5. Konfiguration erstellen

Wir erstellen eine Konfigurationsdatei für das Training mit dem AIR-125 Dataset.

In [ ]:
import yaml
from pathlib import Path

# Definiere Pfade
raw_data_path = str(Path("./data/AIR").absolute())
processed_data_path = str(Path("./data/processed_AIR").absolute())

# Erstelle processed data Verzeichnis
Path(processed_data_path).mkdir(parents=True, exist_ok=True)

# Erstelle Konfiguration
config = {
    'BASE': [''],
    'TOOLBOX_MODE': 'train_and_test',
    'TRAIN': {
        'BATCH_SIZE': 4,
        'EPOCHS': 10,
        'LR': 0.001,
        'MODEL_FILE_NAME': 'AIR_VIRENet_model',
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.0,
            'END': 0.7,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'VALID': {
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.7,
            'END': 1.0,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'TEST': {
        'METRICS': ['MAE', 'RMSE', 'MAPE', 'Pearson'],
        'USE_LAST_EPOCH': False,
        'DATA': {
            'FS': 5,
            'DATASET': 'AIR',
            'DO_PREPROCESS': True,
            'DATA_FORMAT': 'NDCHW',
            'DATA_PATH': raw_data_path,
            'CACHED_PATH': processed_data_path,
            'EXP_DATA_NAME': '',
            'BEGIN': 0.0,
            'END': 1.0,
            'PREPROCESS': {
                'DATA_TYPE': ['Standardized'],
                'LABEL_TYPE': 'Standardized',
                'DO_CHUNK': True,
                'CHUNK_LENGTH': 60,
                'DYNAMIC_DETECTION': False,
                'DYNAMIC_DETECTION_FREQUENCY': 60,
                'CROP_FACE': False,
                'LARGE_FACE_BOX': True,
                'LARGE_BOX_COEF': 1.5,
                'H': 96,
                'W': 96
            }
        }
    },
    'DEVICE': 'cuda:0',
    'NUM_OF_GPU_TRAIN': 1,
    'LOG': {
        'PATH': 'runs/exp'
    },
    'MODEL': {
        'DROP_RATE': 0.2,
        'NAME': 'VIRENet',
        'MODEL_DIR': '',
        'VIRENET': {
            'FRAME_DEPTH': 10
        }
    },
    'INFERENCE': {
        'BATCH_SIZE': 4,
        'EVALUATION_METHOD': 'FFT',
        'MODEL_PATH': ''
    }
}

# Speichere Konfiguration
config_path = Path('./notebook_config.yaml')
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"Konfiguration erstellt: {config_path.absolute()}")
print(f"\nDataset-Pfad: {raw_data_path}")
print(f"Verarbeiteter Dataset-Pfad: {processed_data_path}")
print(f"\nKonfiguration:")
print(f"  - Modell: {config['MODEL']['NAME']}")
print(f"  - Batch Size: {config['TRAIN']['BATCH_SIZE']}")
print(f"  - Epochen: {config['TRAIN']['EPOCHS']}")
print(f"  - Learning Rate: {config['TRAIN']['LR']}")
print(f"  - Device: {config['DEVICE']}")

## 6. Model Training

Jetzt trainieren wir das VIRENet (AIRFlowNet) Modell auf dem AIR-125 Dataset.

**Hinweis:** 
- Das Training kann je nach Hardware mehrere Stunden dauern
- GPU wird empfohlen (CUDA)
- Der erste Run führt Preprocessing durch (DO_PREPROCESS: True)

In [ ]:
# Überprüfe ob das Hauptverzeichnis korrekt ist
import os
import sys

# Falls wir nicht im Hauptverzeichnis sind, wechsle dorthin
repo_root = Path.cwd()
if not (repo_root / 'main.py').exists():
    # Suche nach main.py im Parent-Verzeichnis
    if (repo_root.parent / 'main.py').exists():
        repo_root = repo_root.parent
        os.chdir(repo_root)

print(f"Working Directory: {os.getcwd()}")
print(f"Config File: {Path('./notebook_config.yaml').absolute()}")

# Füge das Repository zum Python-Pfad hinzu
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print("\nStarte Training...")
print("Dies kann mehrere Stunden dauern.\n")

In [ ]:
# Führe Training aus
!python main.py --config_file ./notebook_config.yaml

## 7. Model Testing / Inference

Nach dem Training können wir das Modell auf Testdaten evaluieren.

In [ ]:
# Erstelle Test-Konfiguration
# Kopiere die bestehende Konfiguration und ändere den Modus auf "only_test"

with open('./notebook_config.yaml', 'r') as f:
    test_config = yaml.safe_load(f)

test_config['TOOLBOX_MODE'] = 'only_test'

# Setze den Pfad zum trainierten Modell
# Dies muss nach dem Training aktualisiert werden
model_path = Path('./runs/exp/AIR_VIRENet_model')  # Passe den Pfad an

if model_path.exists():
    # Finde das beste oder letzte Modell
    model_files = list(model_path.glob('*.pth'))
    if model_files:
        latest_model = max(model_files, key=os.path.getctime)
        test_config['INFERENCE']['MODEL_PATH'] = str(latest_model)
        print(f"Verwende Modell: {latest_model}")
    else:
        print("Keine .pth Modell-Dateien gefunden.")
else:
    print(f"Modell-Verzeichnis nicht gefunden: {model_path}")
    print("Bitte führe zuerst das Training aus.")

# Speichere Test-Konfiguration
test_config_path = Path('./notebook_test_config.yaml')
with open(test_config_path, 'w') as f:
    yaml.dump(test_config, f, default_flow_style=False)

print(f"\nTest-Konfiguration erstellt: {test_config_path}")

In [ ]:
# Führe Testing aus
!python main.py --config_file ./notebook_test_config.yaml

## 8. Ergebnisse visualisieren

Lass uns die Trainingsergebnisse und Metriken visualisieren.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Finde die Ergebnisdateien
results_dir = Path('./runs/exp')

if results_dir.exists():
    # Suche nach CSV-Dateien mit Metriken
    csv_files = list(results_dir.rglob('*.csv'))
    
    if csv_files:
        print(f"Gefundene Ergebnisdateien: {len(csv_files)}")
        
        for csv_file in csv_files[:3]:  # Zeige die ersten 3
            print(f"\n{'='*60}")
            print(f"Datei: {csv_file.name}")
            print(f"{'='*60}")
            
            try:
                df = pd.read_csv(csv_file)
                print(f"\n{df.to_string()}")
                
                # Wenn die Datei numerische Daten enthält, plotte sie
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                if len(numeric_cols) > 0:
                    fig, axes = plt.subplots(1, min(len(numeric_cols), 3), figsize=(15, 4))
                    if len(numeric_cols) == 1:
                        axes = [axes]
                    
                    for idx, col in enumerate(numeric_cols[:3]):
                        axes[idx].plot(df[col])
                        axes[idx].set_title(col)
                        axes[idx].set_xlabel('Index')
                        axes[idx].set_ylabel('Value')
                        axes[idx].grid(True)
                    
                    plt.tight_layout()
                    plt.show()
            except Exception as e:
                print(f"Fehler beim Laden von {csv_file.name}: {e}")
    else:
        print("Keine CSV-Ergebnisdateien gefunden.")
else:
    print(f"Ergebnisverzeichnis nicht gefunden: {results_dir}")

## 9. TensorBoard (optional)

Falls TensorBoard-Logs erstellt wurden, können wir diese visualisieren.

In [ ]:
# Lade TensorBoard in Jupyter
%load_ext tensorboard

# Starte TensorBoard
log_dir = './runs/exp'
print(f"Starte TensorBoard mit Log-Verzeichnis: {log_dir}")
%tensorboard --logdir {log_dir}

## 10. Verschiedene Modelle ausprobieren

Du kannst auch andere Modelle ausprobieren, indem du die Konfiguration änderst.

In [ ]:
# Verfügbare Modelle
available_models = {
    'VIRENet': 'AIRFlowNet - Flow-basiertes Modell (empfohlen für Säuglinge)',
    'DeepPhys': 'DeepPhys - Attention-basiertes Modell',
    'EfficientPhys': 'EfficientPhys - Effizientes Modell',
    'Tscan': 'TS-CAN - Temporal Shift Attention Network'
}

print("Verfügbare Modelle:")
print("="*60)
for model_name, description in available_models.items():
    print(f"\n{model_name}:")
    print(f"  {description}")

print("\n" + "="*60)
print("\nUm ein anderes Modell zu verwenden:")
print("1. Ändere config['MODEL']['NAME'] in Zelle 5")
print("2. Führe die Training-Zellen erneut aus")

## 11. Zusammenfassung und nächste Schritte

### Was wir getan haben:
1. Umgebung eingerichtet und Abhängigkeiten installiert
2. AIR-125 Dataset heruntergeladen und extrahiert
3. Konfigurationsdatei erstellt
4. Modell trainiert
5. Modell getestet und evaluiert
6. Ergebnisse visualisiert

### Nächste Schritte:
- **Hyperparameter-Tuning**: Experimentiere mit verschiedenen Lernraten, Batch-Größen, etc.
- **Andere Modelle**: Probiere DeepPhys, EfficientPhys oder TS-CAN aus
- **COHFACE Dataset**: Evaluiere auf dem Erwachsenen-Dataset
- **Optical Flow**: Generiere Optical Flow Inputs für verbesserte Ergebnisse
- **Eigene Videos**: Teste das Modell auf eigenen Säuglingsvideos

### Wichtige Konfigurationsparameter:
```yaml
TRAIN:
  BATCH_SIZE: 4          # Batch-Größe (größer = schneller, aber mehr GPU-Memory)
  EPOCHS: 10             # Anzahl Trainings-Epochen
  LR: 0.001              # Lernrate

PREPROCESS:
  CHUNK_LENGTH: 60       # Länge der Video-Chunks in Frames
  H: 96                  # Höhe der verarbeiteten Frames
  W: 96                  # Breite der verarbeiteten Frames
  CROP_FACE: False       # Gesichtserkennung aktivieren/deaktivieren

MODEL:
  NAME: VIRENet          # Modellname (VIRENet, DeepPhys, EfficientPhys, Tscan)
```

### Referenzen:
- [Paper](https://arxiv.org/pdf/2307.13110.pdf)
- [GitHub Repository](https://github.com/ubicomplab/rPPG-Toolbox)
- [Dataset](https://coe.northeastern.edu/Research/AClab/AIR-125/)

### Support:
Bei Fragen oder Problemen, siehe README.md oder öffne ein Issue auf GitHub.